<a href="https://colab.research.google.com/github/Liao-HsienTing/PL-Repo./blob/main/114_1_HW2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



google sheet：https://docs.google.com/spreadsheets/d/1Blu5BFuKKoRPwLRxnv1sN7MTorHqLjKS0hXLjbIk-Nw/edit?usp=sharing

In [4]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

In [5]:
import pandas as pd
# 在 google 工作表載入 gsheets
gsheets = gc.open_by_url('https://docs.google.com/spreadsheets/d/1Blu5BFuKKoRPwLRxnv1sN7MTorHqLjKS0hXLjbIk-Nw/edit?usp=sharing')

In [6]:
import os
from google import genai
from google.genai import types

# 初始化 Gemini 客戶端

# 優先嘗試從環境變數中讀取 API 金鑰
api_key_from_env = os.environ.get('GEMINI_API_KEY')

try:
    if api_key_from_env:
        # 如果環境變數存在，則使用它來初始化客戶端
        client = genai.Client(api_key=api_key_from_env)
        print("✅ Gemini API 客戶端初始化成功 (使用環境變數)。")
    else:
        # 如果環境變數不存在，則拋出錯誤提醒使用者設定
        raise ValueError("環境變數 'GEMINI_API_KEY' 未設定。")

except Exception as e:
    # 這將捕獲到沒有金鑰或初始化失敗的錯誤
    print(f"❌ Gemini API 客戶端初始化失敗：{e}")
    client = None

❌ Gemini API 客戶端初始化失敗：環境變數 'GEMINI_API_KEY' 未設定。


In [7]:
# Load the data from the Google Sheet
sheet = gsheets.worksheet('工作表1')  # Replace with your sheet name if different
data = sheet.get_all_records()
df = pd.DataFrame(data)

# Convert the '金額' column to numeric, coercing errors
df['金額'] = pd.to_numeric(df['金額'], errors='coerce')

# Drop rows where '金額' is NaN after conversion
df.dropna(subset=['金額'], inplace=True)

# Format the data for Gemini API input
# You can adjust the format based on what you think works best for the model
# For example, a simple list of entries or a summary by category
spending_data_formatted = "我的消費紀錄如下：\n"
for index, row in df.iterrows():
    spending_data_formatted += f"日期: {row['日期']}, 時間: {row['時間']}, 項目: {row['項目']}, 金額: {row['金額']}, 付款方式: {row['付款方式']}\n"

print("已讀取並格式化消費紀錄。")

已讀取並格式化消費紀錄。


In [8]:
# Use Gemini API to analyze spending habits
try:
    model = genai.GenerativeModel('gemini-1.5-flash-latest') # You can choose a different model if needed

    prompt = f"""
    請根據以下的消費紀錄，分析我的消費習慣，並提供一些建議。
    請專注於分析支出的類別、金額分佈，以及可能的節省方式。

    {spending_data_formatted}

    分析與建議：
    """

    print("正在使用 Gemini API 分析消費紀錄...")
    response = model.generate_content(prompt)

    # Extract the analysis from the response
    analysis_summary = response.text
    print("\n--- 消費習慣分析與建議 ---")
    print(analysis_summary)

    # Now you can write this analysis_summary back to a specific cell in your Google Sheet
    # Or display it in the notebook

except Exception as e:
    print(f"❌ Gemini API 分析失敗：{e}")
    analysis_summary = "無法進行消費習慣分析。"

❌ Gemini API 分析失敗：module 'google.genai' has no attribute 'GenerativeModel'


### 設定 Gemini API 金鑰

1.  首先，您需要取得一個 Gemini API 金鑰。如果還沒有，可以在 [Google AI Studio](https://aistudio.google.com/app/apikey) 建立一個。
2.  在 Colab 環境中，點擊左側面板的「🔑 密鑰」圖示。
3.  點擊「新增密鑰」。
4.  在「名稱」欄位輸入 `GEMINI_API_KEY` (請務必使用這個名稱)。
5.  在「值」欄位貼上您的 Gemini API 金鑰。
6.  勾選「在筆記本中顯示這個密鑰」。
7.  點擊「儲存」。

接下來，我會修改程式碼，讓它從 Colab 的密鑰管理器中讀取您的 API 金鑰。

In [9]:
# Used to securely store your API key
from google.colab import userdata
import google.generativeai as genai
import os

# 從 Colab 密鑰管理器中讀取 API 金鑰
try:
    GOOGLE_API_KEY = userdata.get('NTNU_PL_week2')
    if not GOOGLE_API_KEY:
         raise ValueError("請在 Colab 密鑰管理器中設定 'GEMINI_API_KEY'!")

    # 初始化 Gemini API
    genai.configure(api_key=GOOGLE_API_KEY)
    print("✅ Gemini API 客戶端初始化成功。")

except Exception as e:
    print(f"❌ Gemini API 客戶端初始化失敗：{e}")
    # 您可以根據需要決定是否要停止執行或採取其他處理方式
    # 例如： client = None

✅ Gemini API 客戶端初始化成功。


In [10]:
# 從 gsheets 的 All-whiteboard-device 載入 sheets
sheets = gsheets.worksheet('工作表1').get_all_values()
# 將 sheets1 資料載入 pd 的 DataFrame 進行分析
df = pd.DataFrame(sheets[1:], columns=sheets[0])
# 取得最前面的資料
df.head()

,日期,時間,項目,金額,付款方式
0,2000-02-02,02:02,djd,3000,信用卡
1,2020-03-03,07:07,jiodw,3000,現金
2,2020-04-04,07:07,huids,300,現金
3,,,,20,line pay
4,2020-03-03,04:04,f,300,現金


In [11]:
from google.colab import auth
import gspread
from google.auth import default
from datetime import datetime

# 步驟 1: 進行 Google 驗證並連接到試算表
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)
gsheets = gc.open_by_url('https://docs.google.com/spreadsheets/d/1Blu5BFuKKoRPwLRxnv1sN7MTorHqLjKS0hXLjbIk-Nw/edit?usp=sharing')

# 步驟 2: 取得特定的工作表 (Worksheet)
worksheet = gsheets.sheet1  # 或者使用 gsheets.worksheet('工作表名稱')


print("--- 輸入資料 ---")

# 步驟 3: 獲取使用者輸入並進行驗證
while True:
    date_str = input("請輸入日期 (格式：YYYY-MM-DD)：")
    try:
        datetime.strptime(date_str, '%Y-%m-%d')
        break
    except ValueError:
        print("日期格式不正確，請重新輸入。")

while True:
    time_str = input("請輸入時間 (格式：HH:MM)：")
    try:
        datetime.strptime(time_str, '%H:%M')
        break
    except ValueError:
        print("時間格式不正確，請重新輸入。")

while True:
    item_str = input("請輸入項目：")
    if item_str.strip() == "":
        print("項目不能為空，請重新輸入。")
    else:
        break

while True:
    amount_str = input("請輸入金額 (必須是數字)：")
    try:
        amount_float = float(amount_str)
        break
    except ValueError:
        print("金額格式不正確，請重新輸入數字。")

# 新增「付款方式」輸入欄位
payment_method = input("請輸入付款方式：")

# 步驟 4: 將驗證後的資料組合成一行清單
# 請確保您的 Google Sheet 欄位順序與此清單順序一致
new_row = [date_str, time_str, item_str, amount_float, payment_method]

print("\n--- 資料驗證成功 ---")
print(f"輸入的資料為：{new_row}")

# 步驟 5: 附加新資料到 Google 試算表
try:
    worksheet.append_rows([new_row])
    print("\n✅ 資料已成功寫入 Google 試算表！")
except Exception as e:
    print(f"\n寫入資料時發生錯誤：{e}")

--- 輸入資料 ---
請輸入日期 (格式：YYYY-MM-DD)：2020-03-03
請輸入時間 (格式：HH:MM)：09:08
請輸入項目：肯德基
請輸入金額 (必須是數字)：500
請輸入付款方式：line pay

--- 資料驗證成功 ---
輸入的資料為：['2020-03-03', '09:08', '肯德基', 500.0, 'line pay']

✅ 資料已成功寫入 Google 試算表！


In [12]:
try:
    # 讀取試算表中所有的記錄，包含標頭列
    existing_data = worksheet.get_all_records()

    # 將讀取的資料轉換為 Pandas DataFrame
    df = pd.DataFrame(existing_data)

    # 檢查 DataFrame 是否為空
    if df.empty:
        print("試算表中沒有任何記錄，無法計算總計。")
    else:
        # 確保「金額」欄位是數字類型，如果不是，嘗試轉換
        if '金額' in df.columns:
            df['金額'] = pd.to_numeric(df['金額'], errors='coerce')

            # 確保「付款方式」欄位存在
            if '付款方式' in df.columns:
                # 依據「付款方式」分組，計算「金額」的總和
                payment_summary = df.groupby('付款方式')['金額'].sum()

                # ---- 修改列印結果的部分 ----
                print("\n--- 各付款方式的總計金額 ---")
                if payment_summary.empty:
                    print("沒有有效的金額可供加總。")
                else:
                    # 遍歷分組加總的結果並逐一列印
                    for payment_method, total_amount in payment_summary.items():
                        # 檢查是否有 NaN 值，如果有則跳過或顯示為 0
                        if pd.isna(total_amount):
                            print(f"{payment_method}: 0 (無效金額)")
                        else:
                            # 您可以選擇性地格式化金額，例如加上貨幣符號或小數點
                            print(f"{payment_method}: {total_amount:,.0f}") # : ,.0f 格式化為整數並加上千分位逗號
                # ----------------------------

            else:
                print("試算表中未找到「付款方式」欄位。")
        else:
            print("試算表中未找到「金額」欄位。")

except Exception as e:
    print(f"讀取或計算總計時發生錯誤：{e}")



--- 各付款方式的總計金額 ---
line pay: 899
信用卡: 3,000
現金: 3,660


In [13]:
import gradio as gr
import gspread
import pandas as pd
from datetime import datetime
from google.colab import auth
from google.auth import default

# Define the submit_expense function
def submit_expense(date_str, time_str, item_str, amount_float, payment_method):
    try:
        # Authenticate and open the spreadsheet (re-authenticate if needed)
        auth.authenticate_user()
        creds, _ = default()
        gc = gspread.authorize(creds)
        gsheets = gc.open_by_url('https://docs.google.com/spreadsheets/d/1Blu5BFuKKoRPwLRxnv1sN7MTorHqLjKS0hXLjbIk-Nw/edit?usp=sharing')
        worksheet = gsheets.sheet1

        # Validate input (basic validation, can be expanded)
        try:
            datetime.strptime(date_str, '%Y-%m-%d')
            datetime.strptime(time_str, '%H:%M')
            amount = float(amount_float) # Ensure amount is float
        except ValueError:
            return "輸入格式錯誤，請檢查日期、時間或金額。", ""

        if not item_str.strip():
             return "項目不能為空。", ""

        # Prepare and append the new row
        new_row = [date_str, time_str, item_str, amount, payment_method]
        worksheet.append_rows([new_row])

        # Calculate and display the summary
        existing_data = worksheet.get_all_records()
        df = pd.DataFrame(existing_data)

        if df.empty:
            summary_text = "試算表中沒有任何記錄，無法計算總計。"
        else:
            if '金額' in df.columns and '付款方式' in df.columns:
                df['金額'] = pd.to_numeric(df['金額'], errors='coerce')
                payment_summary = df.groupby('付款方式')['金額'].sum()
                summary_text = "--- 各付款方式的總計金額 ---\n"
                if payment_summary.empty:
                    summary_text += "沒有有效的金額可供加總。"
                else:
                    for payment_method, total_amount in payment_summary.items():
                         if pd.isna(total_amount):
                             summary_text += f"{payment_method}: 0 (無效金額)\n"
                         else:
                             summary_text += f"{payment_method}: {total_amount:,.0f}\n"
            else:
                summary_text = "試算表中未找到「金額」或「付款方式」欄位。"

        return "✅ 資料已成功寫入 Google 試算表！", summary_text

    except Exception as e:
        return f"寫入資料時發生錯誤：{e}", ""


# 建立 Gradio 介面
with gr.Blocks(title="日常支出紀錄系統") as demo:
    gr.Markdown("## 歡迎使用日常支出紀錄系統")
    gr.Markdown("請在下方輸入您的消費紀錄，系統會自動寫入 Google 試算表並計算總額。")
    gr.Markdown("---")

    with gr.Row():
        date_input = gr.Textbox(label="日期", placeholder="YYYY-MM-DD", interactive=True)
        time_input = gr.Textbox(label="時間", placeholder="HH:MM", interactive=True)

    with gr.Row():
        item_input = gr.Textbox(label="項目", placeholder="例如：午餐", interactive=True)
        amount_input = gr.Number(label="金額", interactive=True)

    payment_input = gr.Textbox(label="付款方式", placeholder="例如：現金、信用卡", interactive=True)

    submit_button = gr.Button("送出紀錄")

    # 顯示結果的文字框
    output_message = gr.Textbox(label="狀態", lines=2, interactive=False, max_lines=2)
    output_summary = gr.Textbox(label="各付款方式總計", lines=5, interactive=False)

    # 將按鈕與函數關聯
    submit_button.click(
        fn=submit_expense,
        inputs=[date_input, time_input, item_input, amount_input, payment_input],
        outputs=[output_message, output_summary]
    )

demo.launch(share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://f61f90342b8da5e805.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://f61f90342b8da5e805.gradio.live


In [14]:
for m in genai.list_models():
    if 'generateContent' in m.supported_generation_methods:
        print(m.name)

models/gemini-2.5-pro-preview-03-25
models/gemini-2.5-flash-preview-05-20
models/gemini-2.5-flash
models/gemini-2.5-flash-lite-preview-06-17
models/gemini-2.5-pro-preview-05-06
models/gemini-2.5-pro-preview-06-05
models/gemini-2.5-pro
models/gemini-2.0-flash-exp
models/gemini-2.0-flash
models/gemini-2.0-flash-001
models/gemini-2.0-flash-exp-image-generation
models/gemini-2.0-flash-lite-001
models/gemini-2.0-flash-lite
models/gemini-2.0-flash-preview-image-generation
models/gemini-2.0-flash-lite-preview-02-05
models/gemini-2.0-flash-lite-preview
models/gemini-2.0-pro-exp
models/gemini-2.0-pro-exp-02-05
models/gemini-exp-1206
models/gemini-2.0-flash-thinking-exp-01-21
models/gemini-2.0-flash-thinking-exp
models/gemini-2.0-flash-thinking-exp-1219
models/gemini-2.5-flash-preview-tts
models/gemini-2.5-pro-preview-tts
models/learnlm-2.0-flash-experimental
models/gemma-3-1b-it
models/gemma-3-4b-it
models/gemma-3-12b-it
models/gemma-3-27b-it
models/gemma-3n-e4b-it
models/gemma-3n-e2b-it
models

In [15]:
# Load the data from the Google Sheet
sheet = gsheets.worksheet('工作表1')  # Replace with your sheet name if different
data = sheet.get_all_records()
df = pd.DataFrame(data)

# Convert the '金額' column to numeric, coercing errors
df['金額'] = pd.to_numeric(df['金額'], errors='coerce')

# Drop rows where '金額' is NaN after conversion
df.dropna(subset=['金額'], inplace=True)

# Format the data for Gemini API input
spending_data_formatted = "我的消費紀錄如下：\n"
for index, row in df.iterrows():
    # Ensuring all necessary columns exist before accessing them
    date_val = row.get('日期', '未知日期')
    time_val = row.get('時間', '未知時間')
    item_val = row.get('項目', '未知項目')
    amount_val = row.get('金額', '未知金額')
    payer_val = row.get('付款方式', '未知付款方式')

    spending_data_formatted += f"日期: {date_val}, 時間: {time_val}, 項目: {item_val}, 金額: {amount_val}, 付款方式: {payer_val}\n"

print("已讀取並格式化消費紀錄。")

已讀取並格式化消費紀錄。


In [16]:
# Use Gemini API to analyze spending habits
try:
    model = genai.GenerativeModel('gemini-2.5-flash') # You can choose a different model if needed

    prompt = f"""
    請根據以下的消費紀錄，分析我的消費習慣，並提供一些建議。
    請專注於分析支出的類別、金額分佈，以及可能的節省方式。
    請用繁體中文回答。

    {spending_data_formatted}

    分析與建議：
    """

    print("正在使用 Gemini API 分析消費紀錄...")
    response = model.generate_content(prompt)

    # Extract the analysis from the response
    analysis_summary = response.text
    print("\n--- 消費習慣分析與建議 ---")
    print(analysis_summary)

    # Now write this analysis_summary back to a specific cell in your Google Sheet
    # Define the name for the analysis sheet
    analysis_sheet_name = '消費分析報告'

    try:
        # Try to open the analysis sheet
        analysis_sheet = gsheets.worksheet(analysis_sheet_name)
        print(f"\n✅ 已找到現有的工作表分頁：'{analysis_sheet_name}'")
    except gspread.WorksheetNotFound:
        # If the sheet does not exist, create it
        analysis_sheet = gsheets.add_worksheet(title=analysis_sheet_name, rows="100", cols="20")
        print(f"\n✅ 已創建新的工作表分頁：'{analysis_sheet_name}'")

    # Clear existing content (optional, depends on desired behavior)
    analysis_sheet.clear()

    # Write the analysis summary to the sheet
    # Split the summary into lines to write into multiple cells
    lines = analysis_summary.splitlines()
    cell_list = analysis_sheet.range('A1:A' + str(len(lines)))

    for i, line in enumerate(lines):
        cell_list[i].value = line

    analysis_sheet.update_cells(cell_list)

    print(f"\n✅ 消費習慣分析已寫入到工作表分頁：'{analysis_sheet_name}'")


except Exception as e:
    print(f"❌ Gemini API 分析或寫入失敗：{e}")
    analysis_summary = "無法進行消費習慣分析。"

正在使用 Gemini API 分析消費紀錄...

--- 消費習慣分析與建議 ---
好的，根據您提供的消費紀錄，這是一個初步的消費習慣分析與建議：

---

### 消費習慣分析

**1. 整體支出概況：**
*   **總支出金額：** 7539 元 (排除一筆日期與項目空白的20元支出，假設為無效紀錄)
*   **交易筆數：** 10 筆有效交易
*   **時間跨度：** 從 2000 年到 2023 年，但絕大多數交易集中在 2020 年。這顯示您的消費紀錄並非持續性或完整性的，主要集中在某個時期。

**2. 支出類別分佈：**
由於大部分項目描述不夠明確，難以進行精確分類。但從現有資料中，我們可以歸納出以下兩大類：

*   **餐飲：**
    *   金額：850 元 (麥當勞 150 + 麥當勞 200 + 肯德基 500)
    *   佔總支出的比例：約 11.27%
    *   這是唯一明確且可歸類的支出項目，顯示您在外食方面有消費。

*   **其他/無法分類：**
    *   金額：6689 元 (djd 3000 + jiodw 3000 + huids 300 + f 300 + 0 9 + hiew 20 + hvh 60)
    *   佔總支出的比例：約 88.73%
    *   這部分佔了絕大部分的支出，但因為項目描述為代碼或單一字元，無法得知這些支出的實際用途。

**3. 金額分佈特點：**
*   **大額支出集中：** 有兩筆金額高達 3000 元的支出 (djd, jiodw)，這兩筆佔了總支出的近 80%。這代表您的消費中有很大部分是單次性的大筆開銷。
*   **小額支出頻繁：** 扣除兩筆 3000 元的支出，其餘八筆交易金額介於 9 元至 500 元之間，顯示您也有不少小額消費。
*   **兩極化現象：** 消費金額呈現兩極化，要麼是非常大額，要麼是較小額，中間金額的支出較少。

**4. 付款方式：**
*   **現金：** 4 筆 (金額總計 3660 元)
*   **Line Pay：** 5 筆 (金額總計 879 元)
*   **信用卡：** 1 筆 (金額總計 3000 元)
*   您混合使用現金、Line Pay 和信用卡付款。Line Pay 在小額餐